In [ ]:
import os
import librosa
import glob
import json
from pathlib import Path
import numpy as np
import wave
import pandas as pd
from pydub import AudioSegment

In [ ]:
def find_values(id, json_repr):
    results = []
    def _decode_dict(a_dict):
        try:
            results.append(a_dict[id])
        except KeyError:
            pass
        return a_dict
    json.loads(json_repr, object_hook=_decode_dict) # Return value ignored.
    return results

emotion_list = []
paths = list(Path('/audio/').rglob('*.wav'))

for k in range(len(paths)):
    with open('clip_'+str(3601+k)+'.json', 'r',encoding="UTF-8") as f:
        json_data = json.load(f)

    json_repr = json.dumps(json_data['data'])

    start = find_values('script_start', json_repr)
    index = [start.index(x) for x in dict.fromkeys(start)]
    start_list = list(dict.fromkeys(start))

    end = find_values('script_end', json_repr)
    end_list = list(dict.fromkeys(end))

    sound = find_values('sound', json_repr)
    sound_list = []
    for m in range(len(index)):
        sound_list.append(sound[m])

    for j in range(len(sound_list)):
        emotion = list((sound_list[j].values()))[0]
        emotion_list.append(emotion)
    
    for i in range(len(start_list)):
        start = int(start_list[i]/30*1000) # Works in milliseconds
        end = int(end_list[i]/30*1000)
        Audio = AudioSegment.from_wav('clip_'+str(3601+k)+'.wav')
        newAudio = Audio[start:end]
        newAudio.export('clip_' + str(3601+k) + '_' + str(i) + '.wav', format="wav")

In [ ]:
a = []
for i in range(len(emotion_list)):
    if emotion_list[i] == 'happy':
        a.append(int(0))
    if emotion_list[i] == 'surprise':
        a.append(int(1))
    if emotion_list[i] == 'angry':
        a.append(int(2))
    if emotion_list[i] == 'sad':
        a.append(int(3))
    if emotion_list[i] == 'dislike':
        a.append(int(4))
    if emotion_list[i] == 'fear':
        a.append(int(5))
    if emotion_list[i] == 'contempt':
        a.append(int(6))
    if emotion_list[i] == 'neutral':
        a.append(int(7))

In [ ]:
label = np.array(a)
label.reshape(-1,1)
n_values = np.max(a) + 1
oh = np.eye(n_values)[a] # one-hot
np.save('C:/Users/User/Desktop/audio/label', oh)

In [ ]:
def load_audio_file(file_path):
    input_length = 44100*4
    data = librosa.core.load(file_path,sr=44100)[0] #, sr=16000
    if len(data)>input_length:
        data = data[:input_length]
    else:
        data = np.pad(data, (0, max(0, input_length - len(data))), "constant")
    return data
def plot_time_series(data):
    fig = plt.figure(figsize=(14, 8))
    plt.title('Raw wave ')
    plt.ylabel('Amplitude')
    plt.plot(np.linspace(0, 4, len(data)), data)
    plt.show()

In [ ]:
### Augmentation
# changing pitch
def pitch_change(data, sampling_rate):
    pitch_factor = random.uniform(0, 5)
    return librosa.effects.pitch_shift(data, sampling_rate, pitch_factor)

# shifting
def shifting(data):
    sft = random.randrange(5000, 10000, 1000)
    data_roll = np.roll(data, sft)
    return data_roll

# changing speed
def speed_change(data):
    speed_factor = random.uniform(0.5, 1.5)
    return librosa.effects.time_stretch(data, speed_factor)

# white noise
def white_noise(data):
    wn = np.random.randn(len(data))
    data_wn = data + 0.005*wn
    return data_wn

In [ ]:
def features_extractor_aug(file_name): # mfcc feature extract
    audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
    mfccs_audio = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_audio = np.mean(mfccs_audio.T,axis=0)
    return mfccs_scaled_audio